# Introduction to Artificial Neural Networks with Keras

Artificial Neural Networks (ANNs): is a Machine Learning model inspired by the networks of biological neurons found in our brains.

#### Biological Neuron
<img width=500 src="images/biological-neuron.jpg">

Biological neurons produce short electrical impulses called *action potentials* (APs, or just *signals*) which travel along the axons and make the synapses release chemical signals called *neurotransmitters*. When a neuron receives suffient amount of these neurotransmitters within a few milliseconds, it fires its own electrical impulses.

These individual neurons are organized in a vast network of billions, with each neuron typically connected to thousands of other neurons.

### Logical computations with Neurons
<img width=500 src="images/log_comp.png">
